In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import optuna
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
tune = False
import optuna
import warnings
warnings.filterwarnings("ignore")

In [14]:
train_df = pd.read_csv('train.csv', index_col='id')
test_df = pd.read_csv('test.csv', index_col='id')
new_column_names = ['Sex',
 'Length',
 'Diameter',
 'Height',
 'Whole_weight',
 'Whole_weight_1',
 'Whole_weight_2',
 'Shell_weight',
 'Rings']
train_df.columns = new_column_names
test_df.columns = new_column_names[:-1]
df = train_df

In [15]:
one_hot_encoded = pd.get_dummies(train_df['Sex'], prefix='Sex') #pd to do one hot encoded
one_hot_encoded

,Sex_F,Sex_I,Sex_M
id,,,
0,True,False,False
1,True,False,False
2,False,True,False
3,False,False,True
4,False,True,False
...,...,...,...
90610,False,False,True
90611,False,False,True
90612,False,True,False


In [16]:
train_df = pd.concat([train_df, one_hot_encoded], axis=1)
train_df = train_df.drop('Sex', axis=1)

In [17]:
one_hot_encoded_test = pd.get_dummies(test_df['Sex'], prefix='Sex') #pd to do one hot encoded
test_df = pd.concat([test_df, one_hot_encoded_test], axis=1)
test_df = test_df.drop('Sex', axis=1)
test_df

,Length,Diameter,Height,Whole_weight,Whole_weight_1,Whole_weight_2,Shell_weight,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,
90615,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,False,False,True
90616,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,False,False,True
90617,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,False,False,True
90618,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,False,False,True
90619,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,False,True,False
...,...,...,...,...,...,...,...,...,...,...
151021,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500,False,True,False
151022,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050,True,False,False
151023,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650,False,True,False


In [18]:
test_df[['Sex_F','Sex_M','Sex_I']] = test_df[['Sex_F','Sex_M','Sex_I']].astype(int)
train_df[['Sex_F','Sex_M','Sex_I']] = train_df[['Sex_F','Sex_M','Sex_I']].astype(int)

In [19]:
one_hot_encoded = pd.get_dummies(df['Sex'], prefix='Sex')
df = pd.concat([df, one_hot_encoded], axis=1)
df.drop('Sex', axis=1, inplace=True)
df[['Sex_F','Sex_M','Sex_I']] = df[['Sex_F','Sex_M','Sex_I']].astype(int)

In [20]:
display(train_df.info(), train_df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 90615 entries, 0 to 90614
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          90615 non-null  float64
 1   Diameter        90615 non-null  float64
 2   Height          90615 non-null  float64
 3   Whole_weight    90615 non-null  float64
 4   Whole_weight_1  90615 non-null  float64
 5   Whole_weight_2  90615 non-null  float64
 6   Shell_weight    90615 non-null  float64
 7   Rings           90615 non-null  int64  
 8   Sex_F           90615 non-null  int32  
 9   Sex_I           90615 non-null  int32  
 10  Sex_M           90615 non-null  int32  
dtypes: float64(7), int32(3), int64(1)
memory usage: 7.3 MB


None

,Length,Diameter,Height,Whole_weight,Whole_weight_1,Whole_weight_2,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794,0.292391,0.365204,0.342405
std,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221,0.454863,0.481490,0.474517
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000,0.000000,0.000000,0.000000
25%,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000,0.000000,0.000000,0.000000
50%,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000,0.000000,0.000000,0.000000
75%,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000,1.000000,1.000000,1.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000,1.000000,1.000000,1.000000


In [21]:
best_params = {
    'learning_rate': 0.06779327492165323,
    'depth': 10,
    'l2_leaf_reg': 93.91802538530435,
    'bagging_temperature': 0.027132913775990974,
    'random_strength': 0.3510810339166272,
    'border_count': 255,
    'grow_policy': 'Lossguide',
     'task_type': 'GPU',
     'iterations': 1000
    
}

In [22]:
X = train_df.drop(columns=['Rings'])  # Features
y = train_df['Rings']

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
def rmsle(y_ture, y_pred):
    return np.sqrt(mean_squared_log_error(y_ture, y_pred))

In [31]:
tune = True
if tune:
    def objective(trial):
        params = {
    'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
    'depth': trial.suggest_int('depth', 4, 10),
    'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 100),
    'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100),
    'random_strength': trial.suggest_loguniform('random_strength', 0.01, 10),
    'border_count': trial.suggest_int('border_count', 32, 255),
    'grow_policy':  'Lossguide',
    'iterations': 1000,
#     'task_type': 'GPU',
    'eval_metric': 'RMSE',
    'logging_level': 'Silent',
#     'subsample': trial.suggest_float('subsample', 0.1, 1.0),  # Fraction of samples used for fitting the individual base learners
    'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 1.0),  # Fraction of features used for fitting the individual base learners at each level
    'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),  # Minimum number of samples needed in a leaf
    'max_leaves': trial.suggest_int('max_leaves', 10, 100),  # Maximum number of leaves in the trees
    'bootstrap_type': 'Bayesian',  # Method used to sample the data points
    'model_size_reg': trial.suggest_float('model_size_reg', 0.1, 10.0),  # Regularization parameter controlling the complexity of the model
#     'max_bin': trial.suggest_int('max_bin', 100, 500)  # Maximum number of bins used for discretizing continuous features
    }
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
        y_pred = model.predict(X_val)
        rmsle_value = rmsle(y_val, y_pred)
        return rmsle_value
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    best_params=study.best_params
    print('Best params', best_params)

    best_model = CatBoostRegressor(**best_params)
    best_model.fit(X_train, y_train, eval_set=(X_val, y_val), logging_level='Slient')

    y_pred = best_model.predict(X_val)

    rmsle_value = rmsle(y_val, y_pred)
    print('RMSLE with best params:', rmsle_value)

[I 2024-04-10 22:49:27,173] A new study created in memory with name: no-name-0c118202-dc89-42d6-b949-666b4f36f4eb
[I 2024-04-10 22:49:36,598] Trial 0 finished with value: 0.15449557039052886 and parameters: {'learning_rate': 0.111762001121996, 'depth': 7, 'l2_leaf_reg': 2.054159481412976, 'bagging_temperature': 11.34802725735412, 'random_strength': 0.05857613144079829, 'border_count': 49, 'colsample_bylevel': 0.7498488790377715, 'min_child_samples': 37, 'max_leaves': 77, 'model_size_reg': 3.7041243295340656}. Best is trial 0 with value: 0.15449557039052886.
[I 2024-04-10 22:49:41,854] Trial 1 finished with value: 0.15351064485978622 and parameters: {'learning_rate': 0.3921629507135117, 'depth': 9, 'l2_leaf_reg': 47.81325037768975, 'bagging_temperature': 0.024348260025999793, 'random_strength': 0.013724336177712747, 'border_count': 127, 'colsample_bylevel': 0.239875480635014, 'min_child_samples': 47, 'max_leaves': 49, 'model_size_reg': 6.560942844308094}. Best is trial 1 with value: 0.1

KeyboardInterrupt: 

In [28]:
best_model = CatBoostRegressor(**best_params, eval_metric='RMSE')

best_model.fit(X, y)
y_pred = best_model.predict(X_val)

rmsle_value = np.sqrt(mean_squared_log_error(y_val, y_pred))
print('RMSLE with best params:', rmsle_value)

0:	learn: 3.0524512	total: 8.77ms	remaining: 8.76s
1:	learn: 2.9390478	total: 15.2ms	remaining: 7.56s
2:	learn: 2.8365385	total: 20.9ms	remaining: 6.96s
3:	learn: 2.7426898	total: 27ms	remaining: 6.73s
4:	learn: 2.6572404	total: 33.5ms	remaining: 6.67s
5:	learn: 2.5799758	total: 39.3ms	remaining: 6.51s
6:	learn: 2.5099571	total: 46.1ms	remaining: 6.54s
7:	learn: 2.4463251	total: 52ms	remaining: 6.45s
8:	learn: 2.3883423	total: 58.2ms	remaining: 6.41s
9:	learn: 2.3368776	total: 64.2ms	remaining: 6.35s
10:	learn: 2.2905621	total: 70ms	remaining: 6.3s
11:	learn: 2.2483026	total: 76.5ms	remaining: 6.29s
12:	learn: 2.2100839	total: 82.4ms	remaining: 6.26s
13:	learn: 2.1761170	total: 88.3ms	remaining: 6.22s
14:	learn: 2.1452346	total: 94.8ms	remaining: 6.22s
15:	learn: 2.1177219	total: 101ms	remaining: 6.2s
16:	learn: 2.0925391	total: 107ms	remaining: 6.17s
17:	learn: 2.0700642	total: 113ms	remaining: 6.16s
18:	learn: 2.0497934	total: 119ms	remaining: 6.13s
19:	learn: 2.0317401	total: 125ms	